<a href="https://colab.research.google.com/github/kanjitp/CS4225/blob/main/cs4225_a2_colab_KanJitpakdi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task 1: Spark SQL (15m)

In [ ]:
# Setup Spark
# ===============
# Installing Spark needs to be done once each time you re-open this notebook. It should take around 10-30 seconds.
# ===============
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.2-bin-hadoop3.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# After downloading dataset, you should have the files in your Files (click the folder icon in the left sidebar)
!wget -O Products_table.csv https://drive.google.com/uc?id=1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH&export=download #products
!wget -O Sales_table.csv https://drive.google.com/uc?id=1l1fr_s67JjGGsXt3fIz_769pKPZg-jhU&export=download #sales
!wget -O Sellers_table.csv https://drive.google.com/uc?id=1YTTYU5Cwgvau1Z7b1ShmcIhrO3VN-Zhq&export=download #sellers

--2023-04-01 07:44:20--  https://drive.google.com/uc?id=1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH
Resolving drive.google.com (drive.google.com)... 172.253.62.102, 172.253.62.101, 172.253.62.139, ...
Connecting to drive.google.com (drive.google.com)|172.253.62.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ab6p3ngbtar5an2on9cmgujhd9g8fv8s/1680335025000/06948221057362969045/*/1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH?uuid=f06e982f-d973-436c-9393-57d6a308ac28 [following]
--2023-04-01 07:44:22--  https://doc-0g-a0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ab6p3ngbtar5an2on9cmgujhd9g8fv8s/1680335025000/06948221057362969045/*/1FG0rGWSPWALcmFo3feHUF5TK5AP7mMwH?uuid=f06e982f-d973-436c-9393-57d6a308ac28
Resolving doc-0g-a0-docs.googleusercontent.com (doc-0g-a0-docs.googleusercontent.com)... 142.251.16.132, 2607:f8b0:4004:c17::84
Connecting to doc-0g

In [ ]:
# read csv files into dataframes, you can work with the 3 tables after running this code
products_table = spark.read.option('header', True).option('inferSchema', True).csv("/content/Products_table.csv").repartition(1).cache()
sales_table = spark.read.option('header', True).option('inferSchema', True).csv("/content/Sales_table.csv").repartition(1).cache()
sellers_table = spark.read.option('header', True).option('inferSchema', True).csv("/content/Sellers_table.csv").repartition(1).cache()

In [ ]:
products_table

DataFrame[product_id: int, product_name: string, price: int]

In [ ]:
sales_table

DataFrame[order_id: int, product_id: int, seller_id: int, num_of_items_sold: int]

In [ ]:
sellers_table

DataFrame[seller_id: int, seller_name: string, rating: int]

In [ ]:
# add all imports
from pyspark.sql.functions import sum, asc, desc, col

In [ ]:
# (a) Output the top 3 most popular products sold among all sellers [2m]
# Your table should have 1 column(s): [product_name]

top_3_products = (
    sales_table
    .groupBy('product_id')
    .sum('num_of_items_sold')
    .withColumnRenamed('sum(num_of_items_sold)', 'total_items_sold')
    .join(products_table, 'product_id')
    .select('product_name')
    .orderBy(col('total_items_sold').desc())
    .limit(3)
)

top_3_products.show()

+-------------+
| product_name|
+-------------+
|product_51270|
|product_18759|
|product_59652|
+-------------+



In [ ]:
# (b) Find out the total sales of the products sold by sellers 1 to 10 and output the top most sold product [2m]
# Your table should have 1 column(s): [product_name]


from pyspark.sql.functions import col

# filter the sellers for seller 1 to 10
filtered_sellers = sales_table.filter(col('seller_id').between(1, 10))

top_product_for_seller_1_to_10 = (
    filtered_sellers
    .groupBy('product_id')
    .agg(sum('num_of_items_sold').alias('total_items_sold'))
    .join(products_table, 'product_id')
    .select('product_name')
    .orderBy(desc('total_items_sold'))
    .limit(1)
)

top_product_for_seller_1_to_10.show()

+-------------+
| product_name|
+-------------+
|product_36658|
+-------------+



In [ ]:
# (c) Compute the combined revenue earned from sellers where seller_id ranges from 1 to 500 inclusive. [3m]
# Your table should have 1 column(s): [total_revenue]

revenue = (
    sales_table
    .filter(col('seller_id').between(1, 500))
    .join(products_table, 'product_id')
    .withColumn('revenue', col('num_of_items_sold') * col('price'))
    .agg(sum('revenue').alias('total_revenue'))
)

revenue.show()

+-------------+
|total_revenue|
+-------------+
|    160916699|
+-------------+



In [ ]:
# (d) Among sellers with rating >= 4 who have achieved a combined number of products sold >= 3000, find out the top 10 most expensive product sold by any of the sellers. (If there are multiple products at the same price, please sort them in ascending order of product_id) [8m]
# Your table should have 1 column(s): [product_name]
# To get the full mark, your query should not run for more than 1 min

high_rating_sellers = sellers_table.filter(col('rating') >= 4)

# sellers with rating >= 4 who have achieved a combined number of products sold >= 3000
sales_high_rating_sellers = (
    sales_table
    .join(high_rating_sellers, 'seller_id')
    .groupBy('seller_id')
    .agg(sum('num_of_items_sold').alias('total_items_sold'))
    .filter(col('total_items_sold') >= 3000)
)

# For video illustration
sales_high_rating_sellers.show(10)

# sellers and their product that they sold
seller_with_product_sold = (
    sales_high_rating_sellers
    .join(sales_table, 'seller_id')
)

# For video illustration
seller_with_product_sold.show(10)

# top 10 most expensive product sold by any of the sellers.
# If there are multiple products at the same price, please sort them in ascending order of product_id)
top_expensive_products = (
    seller_with_product_sold
    .join(products_table, 'product_id')
    .select('product_name')
    .orderBy(desc('price'), asc('product_id'))
    .limit(10)
)

# Final Result
top_expensive_products.show(10)


+---------+----------------+
|seller_id|total_items_sold|
+---------+----------------+
|    23608|            6285|
|    34610|            4441|
|     1543|            4147|
|    10269|            3071|
|    38703|            3897|
|    35872|            4955|
|    39620|            5147|
|    31478|            3384|
|    16139|            3577|
|    13309|            4448|
+---------+----------------+
only showing top 10 rows

+---------+----------------+--------+----------+-----------------+
|seller_id|total_items_sold|order_id|product_id|num_of_items_sold|
+---------+----------------+--------+----------+-----------------+
|    23608|            6285|  293756|     95179|              673|
|    23608|            6285|  175321|     51037|              531|
|    23608|            6285|  149971|     49213|              840|
|    23608|            6285|  147765|     16749|              898|
|    23608|            6285|  128426|     51717|               39|
|    23608|            6285|   6

## Task 2: Spark ML (10m)

In [2]:
# Setup Spark
# ===============
# Installing Spark needs to be done once each time you re-open this notebook. It should take around 10-30 seconds.
# ===============
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.3.2-bin-hadoop3.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
# After downloading dataset, you should have the files in your Files (click the folder icon in the left sidebar)
!wget -O bank_train.csv https://drive.google.com/uc?id=1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC&export=download #products
!wget -O bank_test.csv https://drive.google.com/uc?id=1EqX4liL5iWbwqyJ_lFaYvYZvgBFwpwSJ&export=download #bank_test

--2023-04-03 02:33:52--  https://drive.google.com/uc?id=1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC
Resolving drive.google.com (drive.google.com)... 74.125.134.113, 74.125.134.139, 74.125.134.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.134.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i36gk5ej68jfuvk62o3hchvn40io6fdr/1680489225000/08487103376102314083/*/1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC?uuid=518284f8-595a-45c4-8da6-ccc579d9cb03 [following]
--2023-04-03 02:33:53--  https://doc-0c-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i36gk5ej68jfuvk62o3hchvn40io6fdr/1680489225000/08487103376102314083/*/1kEP94BfULB3gUMl_IQCg9wuX4IJRajMC?uuid=518284f8-595a-45c4-8da6-ccc579d9cb03
Resolving doc-0c-a8-docs.googleusercontent.com (doc-0c-a8-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to doc-0

In [4]:
bank_train_location = "/content/bank_train.csv"
bank_test_location = "/content/bank_test.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bank_train = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(bank_train_location)

bank_test = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(bank_test_location)

Build ML model to predict whether the customer will subscribe bank deposit service or not. Train the model using training set and evaluate the model performance (e.g. accuracy) using testing set.


*   You can explore different methods to pre-process the data and select proper features
*   You can explore different methods to pre-process the data and select proper features
*   Present the final testing accuracy.

In [5]:
# In this task I will train 3 separate ML algorithm and tune their hyperparameters to find the 
# best model for each of them through hyper parameter tuning

# data preparation (4m)

# Converting string (aka categorial data) to numerical data
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, ChiSqSelector
from pyspark.ml import Pipeline

# List of categorical columns
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Create a list of indexers and encoders for categorical columns
# assignes unique index to each distinct string value
indexers = [StringIndexer(inputCol=column, outputCol=column + "_index") for column in categorical_columns]
# assign index 0 or 1 to the indexed string 
encoders = [OneHotEncoder(inputCol=column + "_index", outputCol=column + "_vec") for column in categorical_columns]

# Define the feature columns to be used in the model
feature_columns = [column + "_vec" for column in categorical_columns] + ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Create a VectorAssembler to combine all feature columns into a single vector
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Create a pipeline to process the data
pipeline = Pipeline(stages=indexers + encoders + [assembler])

# Fit the pipeline to the training data and transform the data
prepared_train_data = pipeline.fit(bank_train).transform(bank_train)
prepared_test_data = pipeline.fit(bank_test).transform(bank_test)

# Selecting most relevant features
# Initialize the ChiSqSelector
# Higher dependence between target label and variables indicate that we can use it to predict the other variable
selector = ChiSqSelector(numTopFeatures=15, featuresCol="features", outputCol="selected_features", labelCol="label")

# Fit the selector to the prepared_train_data
selector_model = selector.fit(prepared_train_data)

# Transform the train and test datasets with the selected features
prepared_train_data = selector_model.transform(prepared_train_data)
prepared_test_data = selector_model.transform(prepared_test_data)

prepared_train_data.show(5)
prepared_test_data.show(5)

+---+----------+-------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+-----+---------+-------------+---------------+-------------+-------------+----------+-------------+-----------+--------------+---------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------------+--------------------+
|age|       job|marital|education|default|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|label|job_index|marital_index|education_index|default_index|housing_index|loan_index|contact_index|month_index|poutcome_index|        job_vec|  marital_vec|education_vec|  default_vec|  housing_vec|     loan_vec|  contact_vec|     month_vec| poutcome_vec|            features|   selected_features|
+---+----------+-------+---------+-------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+-----+---------+-------------

In [10]:
# model building (4m)

from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier,  LogisticRegression, GBTClassifier
from pyspark.sql.functions import sum, asc, desc, col

# Create and train the models
models = {
    'Random Forest': RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=50, maxDepth=10),
    'Logistic Regression': LogisticRegression(featuresCol="features", labelCol="label"),
    'Gradient-Boosted Trees': GBTClassifier(featuresCol="features", labelCol="label", maxIter=50, maxDepth=5)
}

trained_models = {name: model.fit(prepared_train_data) for name, model in models.items()}

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")

# Print result of different models
for name, model in trained_models.items():
    predictions = model.transform(prepared_test_data)
    auc = evaluator.evaluate(predictions)
    accuracy = predictions.filter(predictions.label == predictions.prediction).count() / predictions.count()
    print(f"{name} - AUC: {auc:.4f}, Accuracy: {accuracy:.4f}")


# Result:
# Logistic Regression - AUC: 0.8754, Accuracy: 0.7895
# Random Forest - AUC: 0.9031, Accuracy: 0.8249
# Gradient-Boosted Trees - AUC: 0.8949, Accuracy: 0.8186    

Random Forest - AUC: 0.9031, Accuracy: 0.8249
Logistic Regression - AUC: 0.8754, Accuracy: 0.7895
Gradient-Boosted Trees - AUC: 0.8949, Accuracy: 0.8186


In [ ]:
# Hyper parameter tuning
# Create Random Forest, Logistic Regression, and Gradient-Boosted Trees models
rf = RandomForestClassifier(featuresCol="features", labelCol="label")

# Create a parameter grid for hyperparameter tuning for Random Forest
rf_paramGrid = (
    ParamGridBuilder()
    .addGrid(rf.numTrees, [10, 50, 100])
    .addGrid(rf.maxDepth, [5, 10, 15])
    .addGrid(rf.impurity, ["gini", "entropy"])
    .build()
)


# Create a cross-validator for model selection and hyperparameter tuning
rf_crossval = CrossValidator(
    estimator=rf,
    estimatorParamMaps=rf_paramGrid,
    evaluator=evaluator,
    numFolds=5,
)

# Train the model using the training data and find the best hyperparameters
rf_cv_model = rf_crossval.fit(prepared_train_data)
best_rf = rf_cv_model.bestModel

print("Best NumTrees:", best_rf.getNumTrees)
print("Best MaxDepth:", best_rf.getOrDefault(best_rf.getParam("maxDepth")))
print("Best Impurity:", best_rf.getOrDefault(best_rf.getParam("impurity")))


rf_predictions = best_rf.transform(prepared_test_data).withColumn("rf_prediction", col("prediction"))

# Evaluate the model performance
rf_auc = evaluator.evaluate(rf_predictions)
rf_accuracy = predictions.filter(rf_predictions.label == rf_predictions.rf_prediction).count() / rf_predictions.count()

# Evaluate the model performance
print(f"Random Forest (Tuned) - AUC: {rf_auc:.4f}, Accuracy: {rf_accuracy:.4f}")

In [ ]:
# Create Logistic Regression 
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Create parameter grids for hyperparameter tuning
lr_paramGrid = (
    ParamGridBuilder()
    .addGrid(lr.regParam, [0.01, 0.1, 1.0])
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
    .build()
)


# Create cross-validators for model selection and hyperparameter tuning
lr_crossval = CrossValidator(
    estimator=lr,
    estimatorParamMaps=lr_paramGrid,
    evaluator=evaluator,
    numFolds=5,
)

# Train the models using the training data and find the best hyperparameters
lr_cv_model = lr_crossval.fit(prepared_train_data)

# Get the best models and print the best hyperparameters
best_lr = lr_cv_model.bestModel

print("Best Logistic Regression Parameters:")
print("Best RegParam:", best_lr.getOrDefault(best_lr.getParam("regParam")))
print("Best ElasticNetParam:", best_lr.getOrDefault(best_lr.getParam("elasticNetParam")))

# Make predictions using the testing data
lr_predictions = best_lr.transform(prepared_test_data).withColumn("lr_prediction", col("prediction"))

# Evaluate the model performance
lr_auc = evaluator.evaluate(lr_predictions)
lr_accuracy = lr_predictions.filter(lr_predictions.label == lr_predictions.lr_prediction).count() / lr_predictions.count()

print(f"\nLogistic Regression (Tuned) - AUC: {lr_auc:.4f}, Accuracy: {lr_accuracy:.4f}")

In [ ]:
# Create gradient boosted tree
gbt = GBTClassifier(featuresCol="features", labelCol="label")

# Create parameter grids for hyperparameter tuning
gbt_paramGrid = (
    ParamGridBuilder()
    .addGrid(gbt.maxIter, [10, 50, 100])
    .addGrid(gbt.maxDepth, [3, 5, 7])
    .addGrid(gbt.stepSize, [0.1, 0.5, 1.0])
    .build()
)

# Create cross-validators for model selection and hyperparameter tuning

gbt_crossval = CrossValidator(
    estimator=gbt,
    estimatorParamMaps=gbt_paramGrid,
    evaluator=evaluator,
    numFolds=5,
)

# Train the models using the training data and find the best hyperparameters
gbt_cv_model = gbt_crossval.fit(prepared_train_data)

# Get the best models and print the best hyperparameters
best_gbt = gbt_cv_model.bestModel

print("\nBest Gradient-Boosted Trees Parameters:")
print("Best MaxIter:", best_gbt.getOrDefault(best_gbt.getParam("maxIter")))
print("Best MaxDepth:", best_gbt.getOrDefault(best_gbt.getParam("maxDepth")))
print("Best StepSize:", best_gbt.getOrDefault(best_gbt.getParam("stepSize")))

# Make predictions using the testing data
gbt_predictions = best_gbt.transform(prepared_test_data).withColumn("gbt_prediction", col("prediction"))

# Evaluate the model performance
gbt_auc = evaluator.evaluate(gbt_predictions)
gbt_accuracy = gbt_predictions.filter(gbt_predictions.label == gbt_predictions.gbt_prediction).count() / gbt_predictions.count()

print(f"Gradient-Boosted Trees (Tuned) - AUC: {gbt_auc:.4f}, Accuracy: {gbt_accuracy:.4f}")


In [ ]:
# model evaluation (2m)

# For this task, I will be making a new test data from previous train and test data 
# this is to avoid overfitting of test data during hyperparameter tuningm

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# Combine the train and test datasets
all_data = bank_train.union(bank_test)

# Split the data into train and test
train_data, new_test_data = all_data.randomSplit([0.8, 0.2], seed=42)

# Apply the same data preparation steps as before

prepared_new_test_data = pipeline.fit(new_test_data).transform(new_test_data)
prepared_test_data = selector_model.transform(prepared_test_data)


# Create evaluators for F1 score, precision, and recall
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
precision_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

# Evaluate the models with the new test dataset
best_models = {'Logistic Regression': best_lr, 'Random Forest': best_rf, 'Gradient-Boosted Trees': best_gbt}
for name, model in best_models.items():
    predictions = model.transform(prepared_new_test_data)
    
    auc = evaluator.evaluate(predictions)
    accuracy = predictions.filter(predictions.label == predictions.prediction).count() / predictions.count()
    f1 = f1_evaluator.evaluate(predictions)
    precision = precision_evaluator.evaluate(predictions)
    recall = recall_evaluator.evaluate(predictions)
    
    print(f"{name} - AUC: {auc:.4f}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
